In [22]:
import pandas as pd
import uuid
import chromadb
from chromadb.utils import embedding_functions

In [11]:
knowledge_base =pd.read_csv('../data/processed_knowledge_base.csv')

In [20]:
chroma_client = chromadb.PersistentClient(path="../vectorstore/")
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [42]:
knowledge_base_collection = chroma_client.create_collection(name="knowledge_base", embedding_function=default_ef,  metadata={"hnsw:space": "cosine"})

In [44]:
metadata = [{'section_heading':row[1]['section_heading'],
             'control_heading':row[1]['control_heading'],
             'answer':row[1]['answer'],
             'notes':row[1]['notes']} for row in knowledge_base.iterrows()]

knowledge_base_collection.add(documents=knowledge_base['question'].to_list(), 
                              ids= [str(uuid.uuid4()) for i in range(len(knowledge_base['question']))],
                              metadatas=metadata)
                    

In [82]:
# Function to classify questions based on similarity search
def classify_question(question, threshold=0.2):
    if question is str:
        question = [question]
    answer = knowledge_base_collection.query(query_texts=question, n_results=1)
    if answer['distances'][0][0] < threshold:
        return 'answerable'
    elif answer['distances'][0][0] < 0.5 and answer['distances'][0][0] > threshold:
        return 'ambiguous'
    else:
        return 'unanswerable'

In [84]:
classify_question(question='What is the purpose of the policy?')

'unanswerable'